The method is described in [Intro2Robotics: Connected Components in a Binary Image](https://www.youtube.com/watch?v=ticZclUYy88) and its Python implementation has been illustrated [here](https://zhuanlan.zhihu.com/p/97689424). Basically, we need to interrogate each Px in each row and label them different number if they are not connected and do this row by row. Eventually, we are able to update the EquDict with an Equ list, which should which components are essentially the same.

In [94]:
import pandas as pd
import numpy as np

In [95]:
RawImage = pd.read_table("input_question_4", header = None) 
# Remove auto-assigned col names
RawImage

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,1,1,0,0,1,1
1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0
2,0,1,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,0,1,0
3,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,1,0,1,1
4,1,0,0,1,0,0,0,0,0,0,1,1,1,0,1,1,1,0,1,1
5,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1
6,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1
7,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1
8,0,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,1,1,1,0
9,1,0,1,0,1,0,1,1,1,1,0,0,1,0,1,0,0,0,0,1


In [96]:
nrow, ncol = RawImage.shape
print(nrow, ncol)

10 20


And then we need to start from the very left to the very right, and do this row by row. We should know that:

1. for each Px, we should consider all adjacent Pxs, which makes an adjacent Px set that is always updating. (**Adj set**)
2. we will check whether there are recognisable Pxs in the **Adj set**, and: 1) label them the same number if they are not labelled; 2) if they are already labelled, we add the two numbers to an **Equ set**.
3. Finally, after all Pxs are labelled, we will then update the whole image with our **Equ set**, which produces the desired output.

In [97]:
def Value(coord, mapx):
    x=coord[0]
    y=coord[1]
    return mapx[y][x]
# print(Value((9,19),RawImage))

In [98]:
def AllColouredPx(image):
    AllColouredPx=[]
    nrow_total, ncol_total = image.shape
    for y in range (ncol_total):
        for x in range (nrow_total):
            if Value((x,y), RawImage)==1:
                AllColouredPx+=tuple([(x,y)])
    return AllColouredPx 

In [99]:
def AdjPoints(coord, image, n):
    AllColouredPx0=AllColouredPx(image)
    if coord not in AllColouredPx0:
        return None
    else:
        x=coord[0]
        y=coord[1]
        if n == 4:
            Adj_unconfirmed=[(x,y),(x-1,y),(x+1,y),(x,y+1),(x,y-1)]
        if n == 8:
            Adj_unconfirmed=[(x,y),(x-1,y),(x+1,y),(x,y+1),(x,y-1),
                     (x-1,y+1),(x+1,y-1),(x+1,y+1),(x-1,y-1)]
        Adj_confirmed=[]
        for Px in Adj_unconfirmed:
            if Px in AllColouredPx0:
                Adj_confirmed.append(tuple(Px))
        return Adj_confirmed

In [108]:
# http://www.py3study.com/Article/details/id/6866.html
def BFS(self, node0):
    queue,order = [],[]
    queue.append(node0)
    order.append(node0)
    while queue:
        ongoing = queue.pop()
        order.append(ongoing)
        for new in self:
            if set(new)&set(ongoing) and new not in order and new not in queue:
                queue.append(new)
    return order

In [107]:
AllPoints = AllColouredPx(RawImage)
# Make a dict with all key valued 0.
ValueDict={}
ValueDict=ValueDict.fromkeys(AllPoints,0)
# Labelling and finding EquSet
EquSet=set()
N_label=0
for point in AllPoints:
    adji=ValueExplore(point, RawImage, 4)[0]
    maxi=ValueExplore(point, RawImage, 4)[1]
    mini=ValueExplore(point, RawImage, 4)[2]
    for i in adji:
        if i != 0 and i != N_label:
            EquSet.add((N_label,i))
    if maxi > 0 and mini !=0:
        for Adj in AdjPoints(point, RawImage, 4):
            ValueDict[Adj]=mini
    elif maxi == 0:
        N_label+=1
        for Adj in AdjPoints(point, RawImage, 4):
            ValueDict[Adj]=N_label        
print(len(EquSet),len(ValueDict),len(AllPoints))
print(EquSet)

17 86 86
{(10, 5), (18, 17), (22, 21), (28, 23), (6, 5), (24, 23), (28, 27), (20, 17), (26, 23), (4, 2), (14, 13), (7, 6), (20, 19), (8, 6), (24, 21), (8, 5), (5, 2)}


In [101]:
Adj_trimed=[]
for i in EquSet:
    Adj_trimed.append((BFS(EquSet,i)))

adj_points=[]
for adj in Adj_trimed:
    adj_point = set()
    for adj_line in adj:
        adj_point.add(adj_line[0])
        adj_point.add(adj_line[1])
    adj_points.append(tuple(adj_point))

In [102]:
# Update ValueDict
EquDict=dict()
for x in list(adj_points):
    for y in x:
        EquDict[y] = min(x)
# EquDict

In [103]:
def update(EquDict0, ValueDict0):
    for x in ValueDict0: 
        if ValueDict0[x] in EquDict0:
            ValueDict0[x] = EquDict0[ValueDict0[x]]
    return ValueDict0

In [104]:
updated_image=np.array(RawImage) 
for key in ValueDict:
    updated_image[key[0],key[1]]=ValueDict[key]
pd.DataFrame(updated_image)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,8,8,0,0,0,15,0,18,0,21,21,21,0,0,27,27
1,1,0,5,0,0,0,0,0,15,15,0,0,0,0,21,0,0,26,0,0
2,0,5,5,5,5,5,5,0,14,0,17,0,20,0,0,0,0,0,28,0
3,0,0,0,0,5,5,0,0,0,0,17,0,0,0,23,23,23,0,28,28
4,2,0,0,7,0,0,0,0,0,0,17,17,17,0,23,23,23,0,28,28
5,2,2,7,7,0,9,0,0,0,0,0,0,0,22,0,0,23,23,0,28
6,2,2,0,0,0,0,11,0,0,0,0,19,19,0,0,0,0,23,0,28
7,0,2,0,0,0,11,11,0,0,0,0,19,19,0,0,0,0,23,28,28
8,0,0,6,6,6,0,0,13,0,16,0,0,19,0,0,0,23,23,23,0
9,3,0,6,0,6,0,13,13,16,16,0,0,19,0,24,0,0,0,0,29


In [105]:
# Writing the output
output= open("output_question_4.txt","a")
for row in updated_image:
    allinrow=""
    for i in row:
        allinrow+=str(i)+" "
    output.write (allinrow)
    output.write ("\n")
output.write ("\n")
output.close()

In [45]:
history


import pandas as pd
import numpy as np
RawImage = pd.read_table("input_question_4", header = None) 
# Remove auto-assigned col names
RawImage
nrow, ncol = RawImage.shape
print(nrow, ncol)
def Value(coord, mapx):
    x=coord[0]
    y=coord[1]
    return mapx[y][x]
# print(Value((9,19),RawImage))
def AllColouredPx(image):
    AllColouredPx=[]
    nrow_total, ncol_total = image.shape
    for y in range (ncol_total):
        for x in range (nrow_total):
            if Value((x,y), RawImage)==1:
                AllColouredPx+=tuple([(x,y)])
    return AllColouredPx
def AdjPoints(coord, image, n):
    AllColouredPx0=AllColouredPx(image)
    if coord not in AllColouredPx0:
        return None
    else:
        x=coord[0]
        y=coord[1]
        if n == 4:
            Adj_unconfirmed=[(x,y),(x-1,y),(x+1,y),(x,y+1),(x,y-1)]
        if n == 8:
            Adj_unconfirmed=[(x,y),(x-1,y),(x+1,y),(x,y+1),(x,y-1),
                     (x-1,y+1),(x+1,y-1),(x+1,y+1),(x-1,y-1)]
        Adj_confirmed